In [1]:
class node:
  def __init__(self,freq,symbol=None,left=None,right=None,root=None):
      self.symbol=symbol
      self.freq=freq
      self.left=left
      self.right=right
      self.root=root

#------tree operations------------------
def updateFreq(node):
    node.freq+=1
    if(node.root):
      return updateFreq(node.root)
    else:
      return True


def swap(node):
    if node.root:
      if node.root.left.freq > node.root.right.freq:
        temp=node.root.left
        node.root.left=node.root.right
        node.root.right=temp
      swap(node.root)


def search(i,Root,move=None):
    if Root.left:
      if Root.left.symbol==i:
        return Root.left
      else:
        return search(i,Root.right,'r')
    if Root.right:
      if Root.right.symbol==i:
        return Root.right
      else:
        return search(i,Root.left,'l')
    else:
      if Root.symbol==i:
        return Root
      else:
        if move=='r':   
          return search(i,Root.root.left,'l')
        if move=='l':
          return search(i,Root.root.right,'r')


def addNewItem(i,rootNode):
    newNode=node(1,symbol=i)
    localroot=node(0)
    loc=search("NYT",rootNode)
    code=traverseCode("NYT",rootNode)
    localroot.root=loc.root
    loc.root.left=localroot
    localroot.left=node(0,symbol="NYT")
    localroot.right=newNode
    newNode.root=localroot
    localroot.left.root=localroot
    updateFreq(localroot)
    swap(localroot)
    code+=fixedCode(i)
    return code

#--------coding--------------------------
def traverseCode(i,Root,code='',move=None):
    if Root.left:
      if Root.left.symbol==i:
        code=code+'0'
        return code
      else:
        code=code+'1'
        return traverseCode(i,Root.right,code,move='r')
    if Root.right:
      if Root.right.symbol==i:
        code=code+'1'
        return code
      else:
        code=code+'0'
        return traverseCode(i,Root.left,code,move='l')

    if Root.symbol==i:
        return code
        
    else:
        if move=='r':   
          code=code[:-1]
          code=code+'0'
          return traverseCode(i,Root.root.left,code,move='l')
        if move=='l':
          code=code[:-1]
          code=code+'1'
          return traverseCode(i,Root.root.right,code,move='r')


def fixedCode(alphabet):
    if (ord(alphabet)-96) < 2*r:
      return '{0:05b}'.format(ord(alphabet)-97 )
    else:
      return '{0:04b}'.format(ord(alphabet)-96-r-1 )


def generateTree(text,codingRootNode):
    codedText=''
    for i in text:
      if (codingRootNode.left ==None)&(codingRootNode.right ==None):
        newNode=node(freq=1,symbol=i)
        codingRootNode.right=newNode
        codingRootNode.left=node(0,"NYT")
        codingRootNode.freq=1
        codingRootNode.root=None
        #links to root
        codingRootNode.right.root=codingRootNode
        codingRootNode.left.root=codingRootNode
        #coding
        codedText=fixedCode(i)
        continue
      else:
        try:  
          loc=search(i,codingRootNode)
        except:
          codedText+=addNewItem(i,codingRootNode)
          continue
        else:
          updateFreq(loc)
          swap(loc)
          codedText+=traverseCode(i,codingRootNode)
          continue
    return codedText,codingRootNode
  

#----------------decoding-------------------------
def decodeChar(text,index):
    if index!=0:
      text=text[index:]
    decimal=int(text[:e],2) 
    if decimal < r:
      decimal=int(text[:e+1],2) + 1
      character=chr(decimal+96)
      index=index+e+1
    else:
      decimal=int(text[:e],2)+r+1
      character=chr(decimal+96)
      index=index+e
    return character,index


def traverse(root,text,index):
    text=text[index:]
    for i in text:
      if root.right:
        if i=='1':
          root=root.right
          index+=1
      if root.left:
        if i=='0':
          root=root.left
          index+=1
    return root.symbol,index


def generateDecodeTree(text,decodingRootNode):
    decodedText=''
    index=0
    while(index!=len(text)):
      if (decodingRootNode.left ==None)&(decodingRootNode.right ==None):
        character,pos= decodeChar(text,index)
        decodedText+=character
        index=pos
        newNode=node(freq=1,symbol=character)
        decodingRootNode.right=newNode
        decodingRootNode.left=node(0,"NYT")
        decodingRootNode.freq=1
        decodingRootNode.root=None
        #links to root
        decodingRootNode.right.root=decodingRootNode
        decodingRootNode.left.root=decodingRootNode

      else:
            character,pos=traverse(decodingRootNode,text,index) 
            if character=='NYT':
              character,pos=decodeChar(text,pos)
              index=pos
              addNewItem(character,decodingRootNode)
              decodedText+=character
            else:
              loc=search(character,decodingRootNode)
              index=pos
              updateFreq(loc)
              swap(loc)
              decodedText+=character      

    return decodedText,decodingRootNode
  

In [2]:
#------------------------------execution--------------------------------
textToCode=input("Enter the string: ")

#-------------------------coding---------------------------------

# m=2^e +r

m=26
e=4
r=10

codingRootNode=node(0)
codedText,codingTree=generateTree(textToCode,codingRootNode)
print(f"Text to code:{textToCode}")
print(f"coded string:{codedText}")

#----------------------decoding--------------------------
decodingRootNode=node(0)
decodedText,decodeTree=generateDecodeTree(codedText,decodingRootNode)
print(f"decoded string:{decodedText}")

Enter the string: aardvark
Text to code:aardvark
coded string:00000101000100000110001011010110001010
decoded string:aardvark
